In [1]:
import gym
import pybullet_envs

import os

from huggingface_sb3 import load_from_hub, package_to_hub

from stable_baselines3 import A2C
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.env_util import make_vec_env

import torch 
from torch import nn

In [2]:
env_id = "AntBulletEnv-v0"
env = gym.make(env_id)

s_size = env.observation_space.shape[0]
a_size = env.action_space

pybullet build time: Sep 21 2022 12:00:53


In [5]:
env = make_vec_env(env_id, n_envs=4)

env = VecNormalize(env, norm_obs=True, norm_reward=False, clip_obs=10.)

### Creating the A2C model

In [6]:
model = A2C(policy = "MlpPolicy",
            env = env,
            gae_lambda = 0.9,
            gamma = 0.99,
            learning_rate = 0.00096,
            max_grad_norm = 0.5,
            n_steps = 8,
            vf_coef = 0.4,
            ent_coef = 0.0,
            tensorboard_log = "./tensorboard",
            policy_kwargs=dict(
            log_std_init=-2, ortho_init=False),
            normalize_advantage=False,
            use_rms_prop= True,
            use_sde= True,
            verbose=1)

Using cuda device


### Training the agent

In [7]:
model.learn(2_000_000)

argv[0]=
argv[0]=
argv[0]=
argv[0]=
argv[0]=
argv[0]=
argv[0]=
argv[0]=
Logging to ./tensorboard/A2C_1
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 172      |
|    ep_rew_mean        | 60.5     |
| time/                 |          |
|    fps                | 393      |
|    iterations         | 100      |
|    time_elapsed       | 8        |
|    total_timesteps    | 3200     |
| train/                |          |
|    entropy_loss       | -0.959   |
|    explained_variance | 0.808    |
|    learning_rate      | 0.00096  |
|    n_updates          | 99       |
|    policy_loss        | 6.42     |
|    std                | 0.134    |
|    value_loss         | 19.5     |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 645      |
|    ep_rew_mean        | 281      |
| time/                 |          |
|    fps                | 510      |
|    iter

### Evaluating the agent

In [9]:
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize

# Load the saved statistics
eval_env = DummyVecEnv([lambda: gym.make("AntBulletEnv-v0")])
eval_env = VecNormalize.load("vec_normalize.pkl", eval_env)

#  do not update them at test time
eval_env.training = False
# reward normalization is not needed at test time
eval_env.norm_reward = False

# Load the agent
model = A2C.load("a2c-AntBulletEnv-v0")

mean_reward, std_reward = evaluate_policy(model, env)

print(f"Mean reward = {mean_reward:.2f} +/- {std_reward:.2f}")

Mean reward = 1806.90 +/- 91.63
